# Sharing Secrets with Quantum distribution

In [3]:
from __future__ import annotations

from typing import NoReturn

import numpy as np
from IPython.display import display

from abc import ABCMeta, abstractmethod
from contextlib import contextmanager

In [5]:
ket0 = np.array([[1], [0]])
ket1 = np.array([[0], [1]])
ket_plus = (ket0 + ket1) / np.sqrt(2)
ket_minus = (ket0 - ket1) / np.sqrt(2)

hadamard_operator = 1 / np.sqrt(2) * np.array([[1, 1], [1, -1]])
sigma_x_operator = np.array([[0, 1], [1, 0]])

In [8]:
class Qubit(metaclass=ABCMeta):
    """量子比特抽象类
    """
    @abstractmethod
    def h(self):
        """Hadamard算子
        """
        
    @abstractmethod
    def x(self):
        """是
        """
    
    @abstractmethod
    def measure(self) -> bool:
        """测量
        """
    
    @abstractmethod
    def reset(self):
        """重置
        """

class QuantumDevice(metaclass=ABCMeta):
    """量子设备
    """
    @abstractmethod
    def allocate_qubit(self) -> Qubit:
        """获取量子比特
        """
        pass
    
    @abstractmethod
    def deallocate_qubit(self, qubit: Qubit):
        """释放量子比特
        """
        pass
    
    @contextmanager
    def using_qubit(self):
        """生成量子比特
        """
        qubit = self.allocate_qubit()
        try:
            yield qubit
        finally:
            qubit.reset()
            self.deallocate_qubit(qubit)

def qrng(device: QuantumDevice) -> bool:
    """产生随机数
    """
    with device.using_qubit() as q:
        q.h()
        return q.measure()

class SimulateQubit(Qubit):
    """量子比特模拟器
    """
    def __init__(self):
        self.reset()
        
    def h(self):
        """Hadamard算子
        """
        self.state = hadamard_operator @ self.state
        
    def x(self):
        self.state = sigma_x_operator @ self.state
    
    def measure(self) -> bool:
        """测量
        """
        pr0 = np.abs(self.state[0, 0]) ** 2
        sample = np.random.random() <= pr0
        return bool(0 if sample else 1)
    
    def reset(self):
        """重置
        """
        self.state = ket0.copy()
        
class SingleQubitSimulator(QuantumDevice):
    """单量子比特模拟器
    """
    def allocate_qubit(self) -> Qubit:
        return SimulateQubit()
    
    def deallocate_qubit(self, qubit: Qubit):
        pass

In [11]:
def prepare_classical_message(bit: bool, q: Qubit) -> NoReturn:
    if bit:
        q.x()
        
def prepare_classical_message_plusminus(bit: bool, q: Qubit) -> NoReturn:
    if bit:
        q.x()
    q.h()
        
def eve_measure(q: Qubit) -> bool:
    return q.measure()

def eve_measure_plusminus(q: Qubit) -> bool:
    q.h()
    return q.measure()

def send_classical_bit(device: QuantumDevice, bit: bool) -> NoReturn:
    with device.using_qubit() as q:
        prepare_classical_message(bit, q)
        result = eve_measure(q)
        q.reset()
        assert result == bit

def send_classical_bit_plusminus(device: QuantumDevice, bit: bool) -> NoReturn:
    with device.using_qubit() as q:
        prepare_classical_message_plusminus(bit, q)
        result = eve_measure_plusminus(q)
    assert result == bit